In [ ]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from scipy import ndimage
from PIL import Image
from skimage import color, img_as_ubyte
from tqdm import tqdm
import cv2
from skimage.color import rgb2gray



def detect_and_remove_specular_highlights(image, threshold=240):
    # Converti l'immagine in array numpy
    image_array = np.array(image)

    # Converti in scala di grigi
    gray=rgb2gray(image_array)
    gray = (gray * 255).astype(np.uint8)

    # Crea la maschera per i pixel con valore maggiore del threshold
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Dilatazione della maschera per includere aree vicine
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.dilate(mask, kernel, iterations=2)

    # Rimuove i riflessi speculari
    corrected_image = cv2.inpaint(image_array, mask, 5, cv2.INPAINT_TELEA)

    return corrected_image, mask

def regional_masking_and_inpainting(input_folder, output_folder, discontinuity_threshold=1 * 10**(-2)):
    # Trova tutte le immagini nella cartella, ordinate per nome
    image_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    if not image_files:
        print(f"Nessuna immagine trovata nella cartella {input_folder}.")
        return

    for i, img_name in enumerate(tqdm(image_files, desc=f"Elaborando immagini in {input_folder}")):
        img_path = os.path.join(input_folder, img_name)
        mask_path = img_path.replace('images', 'masks')
        img = Image.open(img_path)
        mask1 = Image.open(mask_path)

        # Converti l'immagine in array numpy
        img_array = np.array(img)

        # Converte in toni di grigio per calcolare i gradienti
        gray_image = color.rgb2gray(img_array)
        J = np.array(gray_image)

        # Definire il kernel derivativo base
        hx = np.array([[-1, 0, 1]])
        hy = np.array([[-1], [0], [1]])

        # Ricavare i gradienti orizzontale e verticale
        Jx = ndimage.correlate(J, hx)
        Jy = ndimage.correlate(J, hy)

        # Identifica la riga centrale
        central_row = Jx[Jx.shape[0] // 2, :]

        # Calcola le differenze tra pixel adiacenti
        diff = np.abs(np.diff(central_row))

        # Trova il primo indice con discontinuità (da sinistra)
        left_bound = np.argmax(diff > discontinuity_threshold)
        # Trova il primo indice con discontinuità (da destra)
        right_bound = len(diff) - np.argmax(diff[::-1] > discontinuity_threshold) - 1

        # Applica la modifica dei pixel sull'immagine a colori
        modified_image = img_array.copy()
        modified_image[:, :left_bound, :] = 16  # Imposta a 16 i pixel a sinistra di left_bound
        modified_image[:, right_bound:, :] = 16  # Imposta a 16 i pixel a destra di right_bound

        # Reset a zero i pixel con tutti i canali pari a 16
        reset_image = modified_image.copy()
        mask = (reset_image[:, :, 0] == 16) & (reset_image[:, :, 1] == 16) & (reset_image[:, :, 2] == 16)
        reset_image[mask] = [0, 0, 0]  # Imposta a [0, 0, 0] i pixel corrispondenti

        # Esegui l'inpainting sull'immagine croppata
        corrected_image, specular_mask = detect_and_remove_specular_highlights(reset_image)

        # Salvataggio delle immagini modificate
        output_img_path = os.path.join(output_folder, 'images', img_name)
        output_mask_path = os.path.join(output_folder, 'masks', os.path.basename(mask_path))

        # Crea le cartelle se non esistono
        os.makedirs(os.path.dirname(output_img_path), exist_ok=True)
        os.makedirs(os.path.dirname(output_mask_path), exist_ok=True)

        # Salva l'immagine modificata
        Image.fromarray(corrected_image).save(output_img_path)

        # Copia la maschera senza modificarla
        mask1.save(output_mask_path)

    print(f"Operazioni di cropping e inpainting completate. Le immagini sono state salvate in {output_folder}.")

def process_all_folders(input_base_folder, output_base_folder, discontinuity_threshold=1 * 10**(-2)):
    subfolders = ['TRAIN', 'VALID', 'TEST']

    for subfolder in subfolders:
        input_folder = os.path.join(input_base_folder, subfolder, 'images')
        output_folder = os.path.join(output_base_folder, subfolder)

        os.makedirs(output_folder, exist_ok=True)
        os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_folder, 'masks'), exist_ok=True)

        regional_masking_and_inpainting(input_folder, output_folder, discontinuity_threshold)

# Esegui la funzione principale
input_base_folder = '/content/drive/MyDrive/CHALLENGE/PolypGen-EIM-24-25_SPLIT_No_organize'
output_base_folder = '/content/drive/MyDrive/CHALLENGE/PolypGen-EIM-24-25_SPLIT_No_organize_REGIONAL MASKING + INPAINTING'
process_all_folders(input_base_folder, output_base_folder)
